In [1]:
import pandas as pd

colunas = ['PROCESSO', 'ROTULO_MANUAL', 'EMENTA_NORM', 'EMENTA_STEM']

df = pd.read_csv('../data/ementas_pre-processadas.csv', header=0, sep=',', quotechar='"', usecols=colunas)

In [2]:
df.head()

,PROCESSO,ROTULO_MANUAL,EMENTA_NORM,EMENTA_STEM
0,00024255420138240081,DAN,apelacao civel acao indenizacao danos morais c...,apelaca civel aca indenizaca dan moral compete...
1,00103872520108240020,CON,apelacao civel acao rescisao contratual cumula...,apelaca civel aca rescisa contrat cumul indeni...
2,00103872520108240020,CON,embargos declaracao apelacao civel inexistenci...,embarg declaraca apelaca civel inexistenc cont...
3,00067390720078240064,PRE,embargos declaracao apelacao civel acordao ape...,embarg declaraca apelaca civel acorda apes ent...
4,00067390720078240064,PRE,embargos declaracao apelacao civel previdencia...,embarg declaraca apelaca civel previdenc priv ...


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

vetor = TfidfVectorizer(ngram_range=(1, 1), max_df=0.95, min_df=0.1)
x = vetor.fit_transform(df['EMENTA_NORM'].values.astype('U'))

In [11]:
len(vetor.vocabulary_)

189

In [5]:
#idf = vetor.idf_
#print(dict(zip(vetor.get_feature_names(), idf)))

In [13]:
vetor.get_feature_names()

['acao',
 'acerca',
 'aclaratorios',
 'acoes',
 'acolhimento',
 'acordao',
 'ademais',
 'adimplemento',
 'advocaticios',
 'afastada',
 'agravo',
 'ainda',
 'alegacao',
 'alegada',
 'analise',
 'apelacao',
 'apelo',
 'apenas',
 'aplicacao',
 'art',
 'artigo',
 'ativa',
 'ausencia',
 'autor',
 'autora',
 'autos',
 'banco',
 'bem',
 'brasil',
 'calculo',
 'caso',
 'causa',
 'cc',
 'civel',
 'civil',
 'cobranca',
 'codigo',
 'coletiva',
 'condenacao',
 'conhecido',
 'conhecimento',
 'consumidor',
 'contradicao',
 'contrato',
 'contratual',
 'controversia',
 'correcao',
 'corte',
 'cpc',
 'cumprimento',
 'danos',
 'data',
 'decisao',
 'declaracao',
 'defesa',
 'demanda',
 'des',
 'desnecessidade',
 'desprovido',
 'desta',
 'deve',
 'dever',
 'direito',
 'dispositivos',
 'embargos',
 'empresa',
 'entendimento',
 'erro',
 'especial',
 'execucao',
 'exegese',
 'existencia',
 'expressa',
 'expurgos',
 'feito',
 'financeira',
 'fixacao',
 'forma',
 'grau',
 'hipotese',
 'honorarios',
 'ii',
 'ii

In [25]:
vetor.vocabulary_

{'apelacao': 15,
 'civel': 33,
 'acao': 0,
 'indenizacao': 88,
 'danos': 50,
 'morais': 116,
 'especial': 68,
 'art': 19,
 'recurso': 157,
 'conhecido': 39,
 'contratual': 44,
 'contrato': 43,
 'bem': 27,
 'razao': 153,
 'condenacao': 38,
 'pagamento': 126,
 'inexistencia': 89,
 'demanda': 55,
 'merito': 112,
 'forma': 77,
 'codigo': 36,
 'processo': 146,
 'civil': 34,
 'parte': 129,
 'autora': 24,
 'mora': 115,
 'inocorrencia': 92,
 'ausencia': 22,
 'aplicacao': 18,
 'lei': 106,
 'exegese': 70,
 'sumula': 178,
 'superior': 179,
 'tribunal': 184,
 'justica': 104,
 'desprovido': 58,
 'termos': 180,
 'desta': 59,
 'corte': 47,
 'precedentes': 139,
 'materia': 110,
 'stj': 176,
 'necessidade': 118,
 'artigo': 20,
 'somente': 175,
 'acerca': 1,
 're': 154,
 'inicial': 91,
 'hipotese': 79,
 'deve': 60,
 'resp': 165,
 'rel': 162,
 'embargos': 64,
 'declaracao': 53,
 'contradicao': 42,
 'obscuridade': 121,
 'omissao': 123,
 'erro': 67,
 'material': 111,
 'inviabilidade': 97,
 'cpc': 48,
 'cas

In [15]:
from sklearn.feature_extraction.text import TfidfTransformer

fidf_transformer = TfidfTransformer()
X_train_tfidf = fidf_transformer.fit_transform(x)
X_train_tfidf.shape



(8117, 189)

In [26]:
X_train_tfidf.toarray()[0]

array([0.1158026 , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.14889116, 0.        , 0.        , 0.        , 0.23029768,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.15488626, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.18933752,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.36397653, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.52329036, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [33]:
y = df['ROTULO_MANUAL'].values

In [34]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, y)

In [35]:
# Naive Bayes 
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

# Predict Class
y_pred = classifier.predict(X_test)

# Accuracy 
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)

TypeError: A sparse matrix was passed, but dense data is required. Use X.toarray() to convert to a dense numpy array.